<h1 style="text-align:center;">MNIST depth experiments 🧀</h1>

In [ ]:
import itertools
import sys
from tqdm import tqdm

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.optim import Adam
from torch.optim import SGD

sys.path.append("..")
from fromage import Fromage

## Get the data and put it in a Pytorch dataloader

In [ ]:
batch_size = 250

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)

## Define the multilayer perceptron architecture

In [ ]:
class SimpleNet(nn.Module):
    def __init__(self, num_layers):
        super(SimpleNet, self).__init__()
        self.layers = nn.ModuleList([nn.Linear(784, 784) for _ in range(num_layers)])
        self.final = nn.Linear(784, 10)
        
    def forward(self, x):
        for i, layer in enumerate(self.layers):
            x = layer(x)
            x = F.relu(x)
        return self.final(x)
    
def getLR(optimizer):
    for param_group in optimizer.param_groups:
        print(f"lr is {param_group['lr']}")
    
net = SimpleNet(5)
for p in net.parameters():
    print(p.size())

## Define a function to train a network

In [ ]:
def train_network(layer_num, epochs, opt, seed, init_lr):
    
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)

    lr_lambda = lambda x: 0.95**x

    model = SimpleNet(layer_num).cuda()
    loss_fn = torch.nn.CrossEntropyLoss()

    if opt == "fromage":
        optim = Fromage(model.parameters(), lr=init_lr)
    elif opt == "SGD":
        optim = SGD(model.parameters(), lr=init_lr, momentum = 0.9)
    elif opt == "SGDwithoutmomentum":
        optim = SGD(model.parameters(), lr=init_lr, momentum = 0.0)
    elif opt == "adam":
        optim = Adam(model.parameters(), lr=init_lr)

    lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optim, lr_lambda)

    print("Training {} layers, {} optim, learning rate {}".format(layer_num, opt, init_lr))

    model.train()
    training_acc = []

    for epoch in range(epochs):

        for batch_idx, (data, target) in enumerate(tqdm(train_loader, file=sys.stdout)):
            data, target = (data.cuda(), target.cuda())

            data = data.view(batch_size,-1)
            y_pred = model(data)
            loss = loss_fn(y_pred, target)

            acc = (target == y_pred.max(dim=1)[1]).sum().item()/batch_size
            training_acc.append(acc)

            model.zero_grad()
            loss.backward()
            optim.step()

        lr_scheduler.step()
    np.save(f'logs/depth/{opt}_layers_{layer_num}_initlr_{init_lr}_epochs100', training_acc)

## Train a bunch of networks at different depths
Be advised that for `epochs = 100` this block will take several days to run on a single GPU.

In [ ]:
opt_list = ["SGD", "fromage", "adam", 'SGDwithoutmomentum']
layer_list_orig = list(range(1,10)) + [10,15,20,25,30,35,40,45,50]

epochs = 100
seed = 0

sgd_lrs = [1.0, 0.1, 0.01]
fromage_lrs = [0.1, 0.01, 0.001]
adam_lrs = [0.01, 0.001, 0.0001]

for opt in opt_list:
    layer_list = layer_list_orig + [] # add empty list to get a copy
    if opt == "SGD" or 'SGDwithoutmomentum': 
        lr_list = sgd_lrs
        layer_list +=[11,12,13,14] # we collect extra data in the interesting regions
    if opt == "fromage": 
        lr_list = fromage_lrs
        layer_list += [27,32,37,42]
    if opt == "adam": 
        lr_list = adam_lrs
        layer_list += [21,22,23,24]
    layer_list.sort()
            
    for layer_num in layer_list:
        for lr in lr_list:
            seed += 1
            train_network( layer_num = layer_num, 
                           epochs = epochs, 
                           opt = opt, 
                           seed = seed, 
                           init_lr = lr )